In [1]:
import utils
from argparse import Namespace

args = Namespace()
args.generate_model="D:\\DDA4210\\facebookopt-1.3b"
args.util_model="D:\\DDA4210\\gpt"
args.prompts_name="lfqa.json"
args.prompt_index = 10
args.use_gpu=True
args.prompt_max_length = None
args.max_new_tokens=200
args.gamma=0.25
args.delta=2.0
args.detection_z_threshold=4.0
args.generation_seed=42
args.use_sampling=True
args.sampling_temp=0.7
args.n_beams=1
args.normalizers=""
args.ignore_repeated_ngrams=False

model, tokenizer, device, pplmodel, ppltokenizer = utils.load_model(args)

original_answer, input_text = utils.load_prompts(args)
print('prompt: ' + input_text)

gpu success
prompt: phil jackson's reputed, patented triangle offense. what are the patterns and reads to the offense and what are the benefits from the perspective of a coach and/or player? what type of players do you need for this offense to be successful?


In [6]:
without_wm, with_wm= utils.generate(input_text, 
                                        args, 
                                        model=model, 
                                        device=device, 
                                        tokenizer=tokenizer)
print('#######################################')
print('generated without watermark: ' + without_wm)
print('#######################################')
print('generated with watermark: ' + with_wm)
rewritten_wm = utils.attack(with_wm)
print('#######################################')
print('rewritten with watermark: ' + rewritten_wm)
refined_wm = utils.refine(with_wm)
print('#######################################')
print('refined with watermark: ' + refined_wm)

#######################################
generated without watermark: 
I don't know anything about this offense, but I'm pretty sure they only run it against the Lakers.
#######################################
generated with watermark: 

The triangle offense was devised by Phil Jackson as a way to attack the weak side of the defense. The triangle consists of a power forward, a center, and a guard. The power forward is the primary scorer while the center is the primary rebounder. The guard is the primary playmaker while the power forward is the primary distributor.

The triangle offense was devised by Phil Jackson as a way to attack the weak side of the defense. The triangle consists of a power forward, a center, and a guard. The power forward is the primary scorer while the center is the primary rebounder. The guard is the primary playmaker while the power forward is the primary distributor.

The triangle offense was devised by Phil Jackson as a way to attack the weak side of the defens

In [3]:
without_wm_detection = utils.detect(without_wm, 
                                    args, 
                                    device=device, 
                                    model = model,
                                    tokenizer=tokenizer)
with_wm_detection = utils.detect(with_wm, 
                                args, 
                                device=device, 
                                model = model,
                                tokenizer=tokenizer)
rewritten_with_wm_detection = utils.detect(rewritten_wm, 
                                        args, 
                                        device=device, 
                                        model = model,
                                        tokenizer=tokenizer)
print('detect finished')

print('#######################################')
print('watermark words:' , with_wm_detection[4][1])

detect finished
#######################################
watermark words: ['The', 'was', 'devised', 'Phil', 'as', 'attack', 'weak', 'side', 'The', 'consists', 'power', 'guard', 'The', 'power', 'primary', 'scorer', 'while', 'primary', 'rebound', 'The', 'guard', 'primary', 'play', 'while', 'power', 'primary', 'distributor', 'The', 'was', 'devised', 'Phil', 'as', 'attack', 'weak', 'side', 'The', 'consists', 'power', 'guard', 'The', 'power', 'primary', 'scorer', 'while', 'primary', 'rebound', 'The', 'guard', 'primary', 'play', 'while', 'power', 'primary', 'distributor', 'The', 'was', 'devised', 'Phil', 'as', 'attack', 'weak', 'side', 'The', 'consists', 'power', 'guard', 'The', 'power', 'primary', 'scorer', 'while', 'primary', 'rebound', 'The', 'guard', 'primary', 'play', 'while']


In [4]:
ppl_without_wm = utils.compute_ppl(without_wm, 
                                    args,
                                    model=pplmodel,
                                    device=device, 
                                    tokenizer=ppltokenizer)
ppl_with_wm = utils.compute_ppl(with_wm,
                                args,
                                model=pplmodel,
                                device=device, 
                                tokenizer=ppltokenizer)
ppl_rewritten_with_wm = utils.compute_ppl(rewritten_wm,
                                args,
                                model=pplmodel,
                                device=device, 
                                tokenizer=ppltokenizer)
print('compute perplexity finished')

compute perplexity finished


In [5]:
analysis = {}
analysis['gamma'] = args.gamma
analysis['delta'] = args.delta
analysis['z_threshold'] = args.detection_z_threshold

analysis['T_with_watermark'] = with_wm_detection[0][1]
analysis['z_with_watermark'] = with_wm_detection[2][1]
analysis['p_with_watermark'] = with_wm_detection[3][1]
analysis['prediction_with_watermark'] = with_wm_detection[6][1]
# analysis['confidence_with_watermark'] = with_wm_detection[7][1]
analysis['ppl_with_watermark'] = ppl_with_wm

analysis['T_without_watermark'] = without_wm_detection[0][1]
analysis['z_without_watermark'] = without_wm_detection[2][1]
analysis['p_without_watermark'] = without_wm_detection[3][1]
analysis['prediction_without_watermark'] = without_wm_detection[6][1]
analysis['ppl_without_watermark'] = ppl_without_wm

analysis['T_attack'] = rewritten_with_wm_detection[0][1]
analysis['z_attack'] = rewritten_with_wm_detection[2][1]
analysis['p_attack'] = rewritten_with_wm_detection[3][1]
analysis['prediction_attack'] = rewritten_with_wm_detection[6][1]
analysis['ppl_attack'] = ppl_rewritten_with_wm

print(analysis)

{'gamma': 0.25, 'delta': 2.0, 'z_threshold': 4.0, 'T_with_watermark': '84', 'z_with_watermark': '5.55', 'p_with_watermark': '1.41e-08', 'prediction_with_watermark': 'True', 'ppl_with_watermark': tensor(132.1930), 'T_without_watermark': '35', 'z_without_watermark': '0.612', 'p_without_watermark': '0.27', 'prediction_without_watermark': 'False', 'ppl_without_watermark': tensor(5.5673), 'T_attack': '3', 'z_attack': '1.09', 'p_attack': '0.138', 'prediction_attack': 'False', 'ppl_attack': tensor(8.0582)}
